In [1]:
import openai

In [2]:
from openai import OpenAI

In [3]:
client = OpenAI()

In [4]:
import os

In [5]:
response = client.chat.completions.create(
    model ='gpt-3.5-turbo-0125',
    messages = [{"role": "user", "content": "Is it too late to join the llm course ?"}]
)

In [6]:
response.choices[0].message.content

"It depends on the specific program and institution. Some LLM programs have specific deadlines for admissions, while others may still be accepting applications on a rolling basis. It's best to check with the program you are interested in to see if they are still accepting new students."

In [7]:
import minsearch

In [8]:
import json

In [9]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [10]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [11]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [12]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [13]:
q = 'the course has already started, can I still enroll?'

In [14]:
index.fit(documents)

In [15]:
boost = { 'question': 3.0 , 'section' : 0.5 }

index.search(
    query=q,
    boost_dict=boost,
    num_results=5,
    filter_dict={'course':'data-engineering-zoomcamp'}   
)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [43]:
response = client.chat.completions.create(
    model ='gpt-4o',
    messages =[{"role": "user", "content": q}]
)

response.choices[0].message.content

"You'll need to check with the specific institution or organization offering the course to determine their enrollment policies. Some courses may allow late enrollment, possibly with certain conditions or deadlines, while others may have strict cut-off dates. Contact the course administrator or admissions office as soon as possible to inquire about your options."

In [16]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results
    

In [24]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [25]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [26]:
query = 'how do I run dlthub?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [27]:
rag(query)

"To run dlthub, ensure that the necessary dependencies are installed. Specifically, you'll need the 'dlt[duckdb]' package. You can install it using the following command:\n\n```bash\n!pip install dlt[duckdb]\n```\n\nAdditionally, if you're running it locally, make sure to have the 'duckdb' package installed separately before using the 'dlt[duckdb]' package."

In [21]:
rag('the course has already started, can I still enroll?')

"Yes, even if you don't register by the start date, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [29]:
rag('what are modules of this course?')

'The provided context does not list the modules of the course.'